In [ ]:
%pip install spotipy --quiet
print("Spotipy installed ✅")

In [ ]:
%pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
SPOTIFY_REDIRECT_URI = "http://127.0.0.1:8080/callback"

if not SPOTIFY_CLIENT_ID or not SPOTIFY_CLIENT_SECRET:
    raise ValueError("❌ Missing Spotify secrets! Create a .env file in the project root with:\nSPOTIFY_CLIENT_ID=your_client_id\nSPOTIFY_CLIENT_SECRET=your_client_secret")

os.environ["SPOTIPY_CLIENT_ID"] = SPOTIFY_CLIENT_ID
os.environ["SPOTIPY_CLIENT_SECRET"] = SPOTIFY_CLIENT_SECRET
os.environ["SPOTIPY_REDIRECT_URI"] = SPOTIFY_REDIRECT_URI

print("Secrets loaded into environment 🔐")

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Scope: read your private & collaborative playlists + liked songs (safe, read-only)
SCOPE = "user-read-private playlist-read-private playlist-read-collaborative user-library-read"

print(f"SPOTIFY_REDIRECT_URI : {SPOTIFY_REDIRECT_URI}")

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    scope=SCOPE,
    redirect_uri=SPOTIFY_REDIRECT_URI,
    open_browser=True,       # ⬅ prevents trying to open port
    show_dialog=True         # ⬅ always show login prompt
))

# ✅ Test 1: Get current user profile
me = sp.current_user()
print("✅ Connected to Spotify as:")
print("  Display name:", me.get("display_name"))
print("  User ID     :", me.get("id"))

# ✅ Test 2: Get first few playlists
print("\n🎵 Fetching first 5 playlists...")
playlists = sp.current_user_playlists(limit=5)

if not playlists["items"]:
    print("No playlists found on this account.")
else:
    for i, pl in enumerate(playlists["items"], start=1):
        print(f"{i}. {pl['name']} (Tracks: {pl['tracks']['total']})")


In [ ]:
!pip install ytmusicapi --quiet
print("ytmusicapi installed ✅")

In [ ]:
from ytmusicapi import YTMusic
print("""
🔐 YOUTUBE MUSIC SETUP REQUIRED (1-time)

Follow these steps:

1️⃣ Open https://music.youtube.com in Chrome (make sure you're logged into your account)

2️⃣ Open Developer Tools (Press F12) → Network tab

3️⃣ Reload the page

4️⃣ Click on request to music.youtube.com [ browse, search, etc ]

5️⃣ Select **Headers** tab → scroll down to **Request Headers**

6️⃣ Copy ALL the request headers (from ":authority" to the last header)

7️⃣ Save them to a file called 'raw_headers.txt' in the project folder

8️⃣ Run the next cell to parse and save them to headers_auth.json
""")

In [ ]:
import json
import os

# Read raw headers from file
RAW_HEADERS_FILE = "raw_headers.txt"

if not os.path.exists(RAW_HEADERS_FILE):
    raise FileNotFoundError(f"❌ '{RAW_HEADERS_FILE}' not found! Please create it with your YouTube Music request headers.")

with open(RAW_HEADERS_FILE, "r", encoding="utf-8") as f:
    raw_headers = f.read()

print(f"✅ Loaded raw headers from '{RAW_HEADERS_FILE}' ({len(raw_headers)} characters)")

In [ ]:
lines = [l.strip() for l in raw_headers.splitlines() if l.strip()]

targets = {
    "authorization": None,
    "cookie": None,
    "user-agent": None,
    "x-goog-authuser": None,
    "x-goog-visitor-id": None,
    "x-origin": None,
    "origin": None,
    "x-youtube-bootstrap-logged-in": None,
    "x-youtube-client-name": None,
    "x-youtube-client-version": None,
}

for i, line in enumerate(lines):
    key_lower = line.lower()
    if key_lower in targets:
        # next non-empty line is the value
        j = i + 1
        while j < len(lines) and not lines[j].strip():
            j += 1
        if j < len(lines):
            targets[key_lower] = lines[j].strip()

headers = {k: v for k, v in targets.items() if v is not None}

with open("headers_auth.json", "w") as f:
    json.dump(headers, f, indent=2)

print("Saved headers_auth.json with keys:", list(headers.keys()))
print("Cookie length:", len(headers.get("cookie", "")))
print("__Secure-3PAPISID in cookie?", "__Secure-3PAPISID=" in headers.get("cookie", ""))

In [ ]:
from ytmusicapi import YTMusic

ytmusic = YTMusic("headers_auth.json")
res = ytmusic.search("Alan Walker", filter="songs")
print("YT Auth Success:", len(res), "results 🎶🎶")

In [ ]:
# STEP 3: High-accuracy Spotify ➜ YouTube Music migration
# - Album-based + fuzzy matching (based on your lookup_song logic, algo=2)
# - Parallel YT lookups per playlist
# - Robust retry & backoff for rate limits / transient network issues
# - Missing tracks logged to per-playlist text files

import time
import re
from typing import List, Dict, Any, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import threading

# Add a lock for thread-safe printing
print_lock = threading.Lock()

# ------------------------ Retry / rate-limiting helpers ------------------------

MAX_BACKOFF = 300  # max backoff per error (seconds)
GLOBAL_COOLDOWN = 0  # shared cooldown applied before each API call


def is_transient_error(e: Exception) -> bool:
    msg = str(e)
    transient_snippets = [
        "429",
        "rate limit",
        "Too Many Requests",
        "Remote end closed connection without response",
        "Connection aborted",
        "temporarily unavailable",
        "timed out",
        "READ_TIMEOUT",
    ]
    msg_lower = msg.lower()
    return any(s.lower() in msg_lower for s in transient_snippets)


def _extract_status_and_retry_after(e: Exception):
    status = getattr(e, "http_status", None)
    retry_after = None
    resp = getattr(e, "response", None)
    if resp is not None:
        status = status or getattr(resp, "status_code", None)
        retry_after = resp.headers.get("Retry-After", None)
    if status is None:
        status = getattr(e, "status", None)
    return status, retry_after


def _apply_cooldown():
    global GLOBAL_COOLDOWN
    if GLOBAL_COOLDOWN > 0:
        wait = GLOBAL_COOLDOWN
        with print_lock:
            print(f"⏳ Global cooldown active: sleeping {wait}s…")
        time.sleep(wait)
        GLOBAL_COOLDOWN = max(0, GLOBAL_COOLDOWN - 5)


def sp_call(func, *args, **kwargs):
    global GLOBAL_COOLDOWN
    for attempt in range(10):
        _apply_cooldown()
        try:
            return func(*args, **kwargs)
        except Exception as e:
            status, retry_after = _extract_status_and_retry_after(e)
            if status == 429 or is_transient_error(e):
                base_wait = int(retry_after) if retry_after else 2 ** attempt
                wait = min(base_wait, MAX_BACKOFF)
                GLOBAL_COOLDOWN = max(GLOBAL_COOLDOWN, wait)
                with print_lock:
                    print(f"[Spotify] Transient / rate error ({status}): {e} → waiting {wait}s…")
                time.sleep(wait)
                continue
            with print_lock:
                print(f"[Spotify] Non-retryable error: {e}")
            raise


def yt_call(func, *args, **kwargs):
    global GLOBAL_COOLDOWN
    for attempt in range(10):
        _apply_cooldown()
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if is_transient_error(e) and attempt < 9:
                wait = min(2 ** attempt, MAX_BACKOFF)
                GLOBAL_COOLDOWN = max(GLOBAL_COOLDOWN, wait)
                with print_lock:
                    print(f"[YTMusic] Transient / rate error: {e} → waiting {wait}s…")
                time.sleep(wait)
                continue
            with print_lock:
                print(f"[YTMusic] Non-retryable error: {e}")
            raise


# ------------------------ High-accuracy YT lookup ------------------------

def lookup_song(
    yt,
    track_name: str,
    artist_name: str,
    album_name: str,
    yt_search_algo: int = 2,
    playlist_name: str = "",
) -> dict:
    track_name_norm = track_name.strip()
    artist_name_norm = artist_name.strip()
    album_name_norm = album_name.strip()

    # 1) Album-based search
    if album_name_norm and artist_name_norm:
        album_query = f"{album_name_norm} by {artist_name_norm}"
        albums = yt_call(yt.search, query=album_query, filter="albums") or []
        for album in albums[:3]:
            try:
                album_details = yt_call(yt.get_album, album["browseId"])
                for track in album_details.get("tracks", []):
                    if track.get("title", "") == track_name_norm:
                        return track
            except Exception as e:
                with print_lock:
                    print(f"Unable to lookup album {album.get('title')} ({e}), continuing...")

    # 2) Song search
    query = f"{track_name_norm} by {artist_name_norm}" if artist_name_norm else track_name_norm
    songs = yt_call(yt.search, query=query, filter="songs") or []

    if not songs:
        raise ValueError(f"No YT Music songs found for query: {query}")

    if yt_search_algo == 0:
        return songs[0]

    if yt_search_algo == 1:
        for song in songs:
            if (
                song.get("title", "") == track_name_norm
                and song.get("artists", [{}])[0].get("name", "") == artist_name_norm
                and song.get("album", {}).get("name", "") == album_name_norm
            ):
                return song
        raise ValueError(
            f"Did not find exact {track_name_norm} by {artist_name_norm} from {album_name_norm}"
        )

    # algo 2: fuzzy matching + videos fallback
    if yt_search_algo == 2:
        for song in songs:
            title = song.get("title", "")
            title_no_brackets = re.sub(r"[\[(].*?[])]", "", title).strip()
            artist0 = song.get("artists", [{}])[0].get("name", "")

            title_ok = (
                (title_no_brackets == track_name_norm)
                or (title_no_brackets in track_name_norm)
                or (track_name_norm in title_no_brackets)
            )
            album_ok = (
                not album_name_norm
                or song.get("album", {}).get("name", "") == album_name_norm
            )
            artist_ok = (
                not artist_name_norm
                or artist0 == artist_name_norm
                or artist_name_norm in artist0
            )

            if title_ok and artist_ok and album_ok:
                return song

        track_name_lower = track_name_norm.lower()
        first_title_lower = songs[0].get("title", "").lower()
        first_artist0 = songs[0].get("artists", [{}])[0].get("name", "")

        if (
            track_name_lower not in first_title_lower
            or (artist_name_norm and first_artist0 != artist_name_norm)
        ):
            prefix = f"[{playlist_name}] '{track_name_norm}'" if playlist_name else f"'{track_name_norm}'"
            with print_lock:
                print(f"   {prefix} - Not found reliably in songs, searching videos...")
            videos = yt_call(
                yt.search,
                query=f"{track_name_norm} by {artist_name_norm}" if artist_name_norm else track_name_norm,
                filter="videos",
            ) or []
            for v in videos:
                v_title = v.get("title", "").lower()
                if (
                    track_name_lower in v_title
                    and (
                        not artist_name_norm
                        or artist_name_norm.lower() in v_title
                    )
                ):
                    with print_lock:
                        print(f"   {prefix} - Found a good candidate video match.")
                    return v

            # # FALLBACK: Search by track name only (ignore artist) to find original version
            # with print_lock:
            #     print(f"   {prefix} - Trying fallback: searching by track name only (ignoring artist)...")
            
            # # Search songs by track name only
            # fallback_songs = yt_call(yt.search, query=track_name_norm, filter="songs") or []
            # for song in fallback_songs:
            #     title = song.get("title", "")
            #     title_lower = title.lower()
            #     title_no_brackets = re.sub(r"[\[(].*?[])]", "", title).strip().lower()
                
            #     # Check if track name matches (ignoring artist)
            #     if (
            #         track_name_lower in title_lower
            #         or track_name_lower in title_no_brackets
            #         or title_no_brackets in track_name_lower
            #     ):
            #         with print_lock:
            #             print(f"   {prefix} - Found match (ignoring artist): {title} by {song.get('artists', [{}])[0].get('name', 'Unknown')}")
            #         return song
            
            # # Also try video search by track name only
            # fallback_videos = yt_call(yt.search, query=track_name_norm, filter="videos") or []
            # for v in fallback_videos:
            #     v_title = v.get("title", "").lower()
            #     if track_name_lower in v_title:
            #         with print_lock:
            #             print(f"   {prefix} - Found video match (ignoring artist): {v.get('title', 'Unknown')}")
            #         return v

            raise ValueError(
                f"Did not find {track_name_norm} by {artist_name_norm} from {album_name_norm} (even in videos)"
            )
        else:
            return songs[0]

    return songs[0]


def lookup_song_video_id(
    yt,
    track_name: str,
    artists: List[str],
    album_name: str,
    playlist_name: str = "",
) -> Optional[str]:
    main_artist = artists[0] if artists else ""
    prefix = f"[{playlist_name}] '{track_name}'" if playlist_name else f"'{track_name}'"
    try:
        song = lookup_song(
            yt=yt,
            track_name=track_name,
            artist_name=main_artist,
            album_name=album_name,
            yt_search_algo=2,
            playlist_name=playlist_name,
        )
        return song.get("videoId")
    except ValueError as e:
        with print_lock:
            print(f"   {prefix} - lookup_song failed: {e}")
        return None
    except Exception as e:
        with print_lock:
            print(f"   {prefix} - Unexpected YT lookup error: {e}")
        return None


# ------------------------ Spotify helpers ------------------------

def get_spotify_playlists() -> List[Dict[str, Any]]:
    playlists = []
    results = sp_call(sp.current_user_playlists)
    while True:
        playlists.extend(results.get("items", []))
        if results.get("next"):
            time.sleep(0.05)
            results = sp_call(sp.next, results)
        else:
            break
    return playlists


def get_spotify_liked_songs() -> List[Dict[str, Any]]:
    """Fetch all liked/saved songs from Spotify."""
    tracks = []
    results = sp_call(sp.current_user_saved_tracks, limit=50)
    while True:
        tracks.extend(results.get("items", []))
        if results.get("next"):
            time.sleep(0.05)
            results = sp_call(sp.next, results)
        else:
            break
    return tracks


def get_spotify_playlist_tracks(playlist_id: str) -> List[Dict[str, Any]]:
    tracks = []
    results = sp_call(sp.playlist_items, playlist_id)
    while True:
        tracks.extend(results.get("items", []))
        if results.get("next"):
            time.sleep(0.05)
            results = sp_call(sp.next, results)
        else:
            break
    return tracks


# ------------------------ Per-track worker (for parallelism) ------------------------

def process_track(idx: int, item: Dict[str, Any], playlist_name: str = "") -> Dict[str, Any]:
    track = item.get("track")
    if not track:
        return {"idx": idx, "video_id": None, "display": "Unknown track"}

    track_name = track.get("name", "Unknown Title")
    artists = [a.get("name", "") for a in track.get("artists", [])]
    try:
        album_name = track.get("album", {}).get("name", "")
    except Exception:
        album_name = ""

    display = f"{track_name} — {', '.join(artists)}"
    with print_lock:
        print(f"  [{idx}] Looking up: {display}")

    video_id = lookup_song_video_id(ytmusic, track_name, artists, album_name, playlist_name)

    return {
        "idx": idx,
        "video_id": video_id,
        "display": display,
    }


# ------------------------ Migration core ------------------------

def safe_filename(name: str) -> str:
    return re.sub(r"[^A-Za-z0-9_.-]+", "_", name)[:80]


def migrate_playlist(spotify_playlist: Dict[str, Any], privacy_status: str = "PRIVATE") -> None:
    name = spotify_playlist.get("name", "Untitled Playlist")
    description = spotify_playlist.get("description") or "Migrated from Spotify"

    with print_lock:
        print(f"\n🎵 Migrating playlist: {name}")

    # 1) Create playlist on YT Music
    yt_playlist_id = yt_call(
        ytmusic.create_playlist,
        title=name,
        description=description,
        privacy_status=privacy_status,
    )
    with print_lock:
        print(f"  → Created YT Music playlist id: {yt_playlist_id}")

    # 2) Fetch tracks from Spotify
    tracks = get_spotify_playlist_tracks(spotify_playlist["id"])
    total_tracks = len(tracks)
    with print_lock:
        print(f"  → Found {total_tracks} tracks on Spotify")

    # 3) Parallel lookup
    results: List[Dict[str, Any]] = []
    max_workers = 1  # Set to 1 for sequential lookups (cleaner logs), or 3-8 for faster parallel lookups

    from concurrent.futures import ThreadPoolExecutor, as_completed
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_track, idx, item, name)
            for idx, item in enumerate(tracks, start=1)
        ]
        for f in as_completed(futures):
            res = f.result()
            results.append(res)

    # Preserve original order
    results.sort(key=lambda x: x["idx"])

    missing_tracks: List[str] = []
    batch_video_ids: List[str] = []

    for res in results:
        idx = res["idx"]
        video_id = res["video_id"]
        display = res["display"]

        if video_id:
            batch_video_ids.append(video_id)
            with print_lock:
                print(f"  [{idx}/{total_tracks}] ✔ {display}")
        else:
            missing_tracks.append(display)
            with print_lock:
                print(f"  [{idx}/{total_tracks}] ❌ Not found: {display}")

        if len(batch_video_ids) >= 40:
            yt_call(ytmusic.add_playlist_items, yt_playlist_id, batch_video_ids)
            with print_lock:
                print(f"  → Added batch of {len(batch_video_ids)} tracks to YT playlist")
            batch_video_ids = []
            time.sleep(0.3)

    # Add remaining
    if batch_video_ids:
        yt_call(ytmusic.add_playlist_items, yt_playlist_id, batch_video_ids)
        with print_lock:
            print(f"  → Added final batch of {len(batch_video_ids)} tracks")

    with print_lock:
        print(f"✅ Finished migrating: {name}")
        if missing_tracks:
            print(f"   ⚠ Not found on YT Music ({len(missing_tracks)} tracks).")
            for t in missing_tracks[:10]:
                print("     -", t)
            if len(missing_tracks) > 10:
                print(f"     ... and {len(missing_tracks) - 10} more")

            # Save full list for manual fixes (in missing/ folder)
            os.makedirs("missing", exist_ok=True)
            fname = f"missing/missing_{safe_filename(name)}.txt"
            with open(fname, "w", encoding="utf-8") as f:
                for t in missing_tracks:
                    f.write(t + "\n")
            print(f"   ↳ Full missing track list saved to: {fname}")
        else:
            print("   🎉 All tracks matched!")


def migrate_liked_songs(liked_tracks: List[Dict[str, Any]], privacy_status: str = "PRIVATE") -> None:
    """Migrate Spotify liked songs to a YouTube Music playlist named 'spot_liked'."""
    name = "spot_liked"
    description = "Liked songs from Spotify"

    with print_lock:
        print(f"\n❤️ Migrating liked songs to playlist: {name}")

    # 1) Create playlist on YT Music
    yt_playlist_id = yt_call(
        ytmusic.create_playlist,
        title=name,
        description=description,
        privacy_status=privacy_status,
    )
    with print_lock:
        print(f"  → Created YT Music playlist id: {yt_playlist_id}")

    total_tracks = len(liked_tracks)
    with print_lock:
        print(f"  → Found {total_tracks} liked songs on Spotify")

    # 2) Parallel lookup (reuse process_track)
    results: List[Dict[str, Any]] = []
    max_workers = 1

    from concurrent.futures import ThreadPoolExecutor, as_completed
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_track, idx, item, name)
            for idx, item in enumerate(liked_tracks, start=1)
        ]
        for f in as_completed(futures):
            res = f.result()
            results.append(res)

    # Preserve original order
    results.sort(key=lambda x: x["idx"])

    missing_tracks: List[str] = []
    batch_video_ids: List[str] = []

    for res in results:
        idx = res["idx"]
        video_id = res["video_id"]
        display = res["display"]

        if video_id:
            batch_video_ids.append(video_id)
            with print_lock:
                print(f"  [{idx}/{total_tracks}] ✔ {display}")
        else:
            missing_tracks.append(display)
            with print_lock:
                print(f"  [{idx}/{total_tracks}] ❌ Not found: {display}")

        if len(batch_video_ids) >= 40:
            yt_call(ytmusic.add_playlist_items, yt_playlist_id, batch_video_ids)
            with print_lock:
                print(f"  → Added batch of {len(batch_video_ids)} tracks to YT playlist")
            batch_video_ids = []
            time.sleep(0.3)

    # Add remaining
    if batch_video_ids:
        yt_call(ytmusic.add_playlist_items, yt_playlist_id, batch_video_ids)
        with print_lock:
            print(f"  → Added final batch of {len(batch_video_ids)} tracks")

    with print_lock:
        print(f"✅ Finished migrating liked songs to: {name}")
        if missing_tracks:
            print(f"   ⚠ Not found on YT Music ({len(missing_tracks)} tracks).")
            for t in missing_tracks[:10]:
                print("     -", t)
            if len(missing_tracks) > 10:
                print(f"     ... and {len(missing_tracks) - 10} more")

            # Save full list for manual fixes (in missing/ folder)
            os.makedirs("missing", exist_ok=True)
            fname = f"missing/missing_{safe_filename(name)}.txt"
            with open(fname, "w", encoding="utf-8") as f:
                for t in missing_tracks:
                    f.write(t + "\n")
            print(f"   ↳ Full missing track list saved to: {fname}")
        else:
            print("   🎉 All liked songs matched!")

In [ ]:
# ------------------------ Main interactive flow (with checkboxes) ------------------------

from ipywidgets import Checkbox, VBox, HBox, Button, Label, Output, Layout
from IPython.display import display

all_playlists = get_spotify_playlists()

# Fetch liked songs count
print("Fetching liked songs count...")
liked_songs = get_spotify_liked_songs()
liked_songs_count = len(liked_songs)
print(f"Found {liked_songs_count} liked songs on Spotify")

# Create output with scrollable container
from IPython.display import display, HTML

# Inject CSS for text wrapping in output
display(HTML("""
<style>
.migration-output-wrap pre,
.migration-output-wrap .output_text,
.migration-output-wrap .output_subarea {
    white-space: pre-wrap !important;
    word-wrap: break-word !important;
    overflow-wrap: break-word !important;
}
</style>
"""))

out = Output()
out.add_class('migration-output-wrap')

# Checkbox for liked songs
liked_songs_cb = Checkbox(
    description=f"❤️ Liked Songs ({liked_songs_count} tracks) → will create 'spot_liked' playlist",
    value=False,
    indent=False,
    style={'description_width': 'initial'}
)

if not all_playlists and liked_songs_count == 0:
    print("No Spotify playlists or liked songs found on this account.")
else:
    # Build one checkbox per playlist
    checkboxes = []
    for i, pl in enumerate(all_playlists, start=1):
        label = f"{i:2d}. {pl['name']}  ({pl['tracks']['total']} tracks)"
        cb = Checkbox(description=label, value=False, indent=False)
        checkboxes.append(cb)

    select_all_cb = Checkbox(description="Select ALL playlists", value=False, indent=False)

    def on_select_all_change(change):
        if change["name"] == "value":
            for cb in checkboxes:
                cb.value = change["new"]

    select_all_cb.observe(on_select_all_change)

    migrate_btn = Button(
        description="Start migration",
        button_style="success",
        tooltip="Migrate selected playlists from Spotify to YouTube Music",
    )

    def on_migrate_clicked(b):
        with out:
            out.clear_output()
            selected_indices = [i for i, cb in enumerate(checkboxes) if cb.value]
            migrate_liked = liked_songs_cb.value

            if not selected_indices and not migrate_liked:
                print("No playlists selected, aborting.")
                return

            total_items = len(selected_indices) + (1 if migrate_liked else 0)
            print(f"\n🚀 Migrating {total_items} item(s) with high-accuracy lookup…")
            
            # Migrate liked songs first if selected
            if migrate_liked:
                print("\n❤️ Migrating Liked Songs → 'spot_liked' playlist...")
                # Create a fake playlist dict for liked songs
                liked_playlist = {
                    "id": "__liked_songs__",
                    "name": "spot_liked",
                    "description": "Liked songs from Spotify",
                }
                migrate_liked_songs(liked_songs, privacy_status="PRIVATE")
            
            # Migrate selected playlists
            selected_playlists = [all_playlists[i] for i in selected_indices]
            for pl in selected_playlists:
                migrate_playlist(pl, privacy_status="PRIVATE")
            
            print("\n🎬 Migration run complete.")

    migrate_btn.on_click(on_migrate_clicked)

    print("Select the playlists you want to migrate:\n")
    # Wrap output in a scrollable HTML container
    from ipywidgets import HTML as HTMLWidget
    
    scroll_container = VBox(
        [out],
        layout=Layout(
            max_height='500px',
            width='100%',
            overflow_y='auto',
            border='1px solid #555',
            padding='10px',
        )
    )
    
    ui = VBox([
        liked_songs_cb,  # Liked songs checkbox at the top
        Label("─" * 50),  # Separator
        select_all_cb,
        VBox(checkboxes),
        migrate_btn,
    ])

    display(ui, scroll_container)
